In [1]:
# library
import numpy
numpy.float = numpy.float64
numpy.int = numpy.int_
import numpy as np
import open3d as o3d
import os.path as osp

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# config
BASE_DIR = '/home/laojk/Code/Dataset/iPad-Stray-Scanner/T'
INTRINSIC = 'odom2colmap/cameras.txt'
EXTRINSIC = 'odom2colmap/images.txt'
POINTS = 'odom2colmap/points3D.txt'

In [3]:
# helper functions
def quaternion_to_rotation_matrix(quaternion):
    qx, qy, qz, qw = quaternion
    rotation_matrix = np.array([[1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
                               [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
                               [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2]])
    return rotation_matrix

def create_transform_matrix(data):
    qw, qx, qy, qz, x, y, z = data
    rotation_matrix = quaternion_to_rotation_matrix([qx, qy, qz, qw])
    transform_matrix = np.eye(4)
    transform_matrix[0:3, 0:3] = rotation_matrix
    transform_matrix[0:3, 3] = [x, y, z]
    transform_matrix[3, 3] = 1
    return transform_matrix

def generate_gradient_color(start_color, end_color, length):
    start_color = np.array(start_color)
    end_color = np.array(end_color)
    color_range = end_color - start_color
    step_size = 1 / (length - 1)

    gradient_color = lambda step: (start_color + color_range * step * step_size) / 255.0
    return gradient_color

In [4]:
# read camera intrinsic
camera_matrix = np.zeros((3, 3))
image_h, image_w = 0, 0
with open(osp.join(BASE_DIR, INTRINSIC), 'r') as f:
    for line in f.readlines():
        if line.startswith('#'):
            continue
        line = line.split()
        if line[0] == '1':
            camera_matrix[0, 0] = float(line[4])
            camera_matrix[1, 1] = float(line[4])
            camera_matrix[0, 2] = float(line[5])
            camera_matrix[1, 2] = float(line[6])
            camera_matrix[2, 2] = 1
            image_w = int(line[2])
            image_h = int(line[3])
            break
camera_matrix = o3d.camera.PinholeCameraIntrinsic(image_w, image_h, camera_matrix)

In [5]:
# read camera extrinsic
extrinsic = []
with open(osp.join(BASE_DIR, EXTRINSIC), 'r') as f:
    line_count = 0
    for line in f.readlines():
        if line.startswith('#'):
            continue
        line_count += 1
        if line_count % 2 == 0:
            continue
        line = line.split()
        extrinsic.append(create_transform_matrix([float(i) for i in line[1:8]]))
extrinsic = np.stack(extrinsic)

In [6]:
# read points
points = []
with open(osp.join(BASE_DIR, POINTS), 'r') as f:
    for line in f.readlines():
        if line.startswith('#'):
            continue
        line = line.split()
        points.append([np.float64(i) for i in line[1:4]])
points = np.stack(points)

In [7]:
# visualize
base_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2, origin=[0, 0, 0])

cameras = []
color = generate_gradient_color([208, 16, 76], [46, 169, 223], len(extrinsic))
for i in range(len(extrinsic)):
    pose = extrinsic[i]
    camera = o3d.geometry.LineSet.create_camera_visualization(camera_matrix, pose, scale=1)
    camera.paint_uniform_color(color(i))
    cameras.append(camera)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.paint_uniform_color([0.1, 0.9, 0.1])

o3d.visualization.draw_geometries([base_frame, pcd] + cameras)